In [2]:
import tensorflow as tf
from __future__ import print_function
import numpy as np


/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## A1 Q1
Euclidean distance function, vectorized    

In [29]:
def eeuclideanDistance(mat1, mat2):
    n1 = mat1.shape[0]
    n2 = mat2.shape[0]
    l2_mat = tf.Variable(tf.zeros([n1, n2]))
    
    l2_mat = tf.reduced_sum(tf.square(tf.subtract(mat1, mat2)))
    print(l2_mat.eval())
    return l2_mat
    
with tf.Session() as sess:
    mat1_ = tf.constant([[3. , 2.],[3., 2.]]) #n1 by d 2 by 2
    mat2_ = tf.constant([[2., 2.], [2., 2.], [ 2., 2.]]) #n2 by d 3 by 2

    l2_ = euclideanDistance(mat1_, mat2_) #n1 by n2

AttributeError: module 'tensorflow' has no attribute 'reduced_sum'

In [26]:
def test():
    A = tf.constant([[1, 1], [2, 2], [3, 3]])
    print(A.shape)
    r = tf.reduce_sum(A*A, 1)
    print(r.eval())
    r = tf.reshape(r, [-1, 1])
    print(r.eval())

    
with tf.Session() as sess:
    test()


(3, 2)
[ 2  8 18]
[[ 2]
 [ 8]
 [18]]


In [59]:
def euclideanDistance(x, z):
    '''
    We vectorized the operation with matrix operations
    the ||x - z ||^2 is = Sum(x - z)^2 = Sum from 1 to D
    of (x^2 - 2xz + z^2)
    
    x^2 = x^T times x
    z^2 = z^T times z
    we can find the sum of each row by using the reduce sum function
    
    sum of -2xz 
    we can use matrix multiplication, x times z^T to obtain xz,
    then multiply by -2
    '''
    n1 = x.shape[0]
    n2 = z.shape[0]    
    
    x_squared = tf.matmul(x, tf.transpose(x))
    z_squared = tf.matmul(z, tf.transpose(z))
    print(x_squared.eval())
    x_2_sum = tf.reduce_sum(x_squared, 1)
    z_2_sum = tf.reduce_sum(z_squared, 1)
    
    x_2_sum = tf.reshape(x_2_sum, [-1, 1])
    z_2_sum = tf.reshape(z_2_sum, [-1, 1])
    
    xz = tf.matmul(x, tf.transpose(z))
    print(xz.eval())
    minus_2xz = tf.scalar_mul(-2, xz)
    #print(minus_2xz.eval())
 
    x_2_tile = tf.tile(tf.expand_dims(x_2_sum, 1),
                      [n1, n])
    print(x_2_tile.eval())
    
    
    #print(x_squared.eval())
    #print(x_2_sum.eval())

def testi():
    A = tf.constant([[1, 1], [2,2], [3, 3]])
    B = tf.constant([[1, 1], [2, 2]])
    euclideanDistance(A, B)
    
with tf.Session() as sess1:
    testi()
    
    


[[ 2  4  6]
 [ 4  8 12]
 [ 6 12 18]]
[[ 2  4]
 [ 4  8]
 [ 6 12]]


ValueError: Shape must be rank 2 but is rank 1 for 'Tile_1' (op: 'Tile') with input shapes: [3,1], [1].

## A1 Q2
### part 1

| x1^T | x2^T | .... |
top k closest in x1^T indices are the neighbours


yˆ(x∗) = YTr∗, where r∗ = [r1,...,rN],rn = 1/k
0, otherwise.




In [81]:
#given starter code
import numpy as np
import matplotlib.pyplot as plt

# my code part1 responsibility
def nearestIndices(_dist_mat, _k):
    with tf.Session() as sess:
        #_dist_mat = tf.constant([ [4, 9], [16, 25] ], tf.int32)
        nearest_k = tf.nn.top_k(tf.negative(_dist_mat), _k)
        #print(_dist_mat.eval())
        print(nearest_k.indices.eval())
        return nearest_k.indices

def responsibility(_data, _neighbours, _k, _n ):
    respon = np.zeros(_n) #total number of data
    #find intersection of _data and _neighbours
    #assign the _neighbours indices in respon to be 1/k
    
    

#unit testing
dist_mat = tf.constant([ [4, 9], [16, 25] ], tf.int32)
topk = nearestIndices(dist_mat, 2)



[[0 1]
 [0 1]]


In [ ]:
np.random.seed(521)
Data = np.linspace(1.0 , 10.0 , num =100) [:, np. newaxis]
Target = np.sin( Data ) + 0.1 * np.power( Data , 2) \
         + 0.5 * np.random.randn(100 , 1)
randIdx = np.arange(100)
np.random.shuffle(randIdx)
trainData, trainTarget  = Data[randIdx[:80]], Target[randIdx[:80]]
validData, validTarget = Data[randIdx[80:90]], Target[randIdx[80:90]]
testData, testTarget = Data[randIdx[90:100]], Target[randIdx[90:100]]

def knn(_data, _target, _k):
    #nearest indices
    #responsbility
    #maximum responsibility
    
    